Bisma Khalid

19I-1797

AI - K


Assignment 3

In [1]:
import os
import time
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision,Accuracy

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf

In [2]:
H=256
W=256
dim=(H,W)


Defining the Structure of the Model

In [3]:
#This convolution part will also be included in encoder and decoder part of the network
def Conv_Block(input,num_of_filter):
    x=Conv2D(num_of_filter,3,padding="same")(input)
    x=BatchNormalization()(x)
    x=Activation('ReLU')(x)

    x=Conv2D(num_of_filter,3,padding="same")(input)
    x=BatchNormalization()(x)
    x=Activation('ReLU')(x)
  
    return x


def Encoder(input,num_of_filter):
    x = Conv_Block(input, num_of_filter) #Skip connection
    p = MaxPool2D((2, 2))(x)  #Feature selection
    return x, p

def Decoder(input, skip_features, num_of_filter):
    x = Conv2DTranspose(num_of_filter, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = Conv_Block(x, num_of_filter)
    return x

Customized Encoder - Decoder Model

In [4]:
def Model_Build(input_shape):
    inputs=Input(input_shape)
    #as you go down in the encoder,resolution decreases , number of filters doubles
    s1,p1=Encoder(inputs,16)
    s2,p2=Encoder(p1,32)
    s3,p3=Encoder(p2,64)
    s4,p4=Encoder(p3,128)
    s5,p5=Encoder(p4,256)
    #s6,p6=Encoder(p5,128)

    b1=Conv_Block(p5,512)

    d1 = Decoder(b1, s5, 256) 
    d2 = Decoder(d1, s4, 128) 
    d3 = Decoder(d2, s3, 64)
    d4 = Decoder(d3, s2, 32)
    d5 = Decoder(d4, s1, 15)
#     d6 = Decoder(d5, s1, 4)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d5)  #our data is in grayscale, gives 0,1 output

    model = Model(inputs, outputs, name="Encoder-Decoder")
    return model


In [5]:
input_shape = (256, 256, 3)
model = Model_Build(input_shape)
model.summary()

Model: "Encoder-Decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 256, 256, 16  64         ['conv2d_1[0][0]']               
 rmalization)                   )                                                   

In [6]:
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32
lr = 0.01
num_epochs = 15

Metrices

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [8]:
model_weights = os.path.join("/kaggle/working/", "Increased_depth_Feature1.h5")
csv_weights = os.path.join("/kaggle/working/", "Increased_depth_Feature1.csv")
 

callbacks = [
    ModelCheckpoint(model_weights, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.00001, patience=2, min_lr=1e-7, verbose=1),
    CSVLogger(csv_weights),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=False),
    ]

Data Loading

In [9]:
# # path="/content/drive/MyDrive/FYP/Dataset/PNG_Slices_Segmented"
path2="/kaggle/input/kidneykits19/PNG_Slices_Segmented/PNG_Slices_Segmented/"
#path1="/kaggle/input/feature-canny2/Feature_2/Feature_2/Feature_2/"
#path1="/kaggle/input/colored/DATA-2/"
path1="/kaggle/input/feature-selection/Feature_1/Feature_1/feature/"

In [10]:
def load_data(file_path1,file_path2):
    #Loading the image files
    images = sorted(glob(f"{file_path1}/*/*.jpg"))
    segmentations=sorted(glob(f"{file_path2}/*/Segmentation/*.png"))
    print(len(images),len(segmentations))


  # #Spliting the data into training and testing using builtin libraries
    split=0.2
    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split, random_state=42)
    train_y, valid_y = train_test_split(segmentations, test_size=split, random_state=42)

    return (train_x, train_y), (valid_x, valid_y)
 


def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x=cv2.resize(x, dim)
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x=cv2.resize(x, dim)
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(x, y, batch=10):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [11]:
(train_x, train_y), (valid_x, valid_y) = load_data(path1,path2)
print(len(train_x),len(train_y))
print(len(valid_x),len(valid_y))
train_x, train_y = shuffling(train_x, train_y)

43098 43098
34478 34478
8620 8620


In [12]:
batch_size = 32
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

Model Training

In [13]:
# model = Model_Build((512, 512, 3))
# metrics = [dice_coef, iou, Recall(), Precision(),Accuracy()]
mse = tf.keras.losses.MeanSquaredError()
metrics = [dice_coef, iou,Accuracy()]
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

In [14]:
# model.fit(train_dataset,epochs=num_epochs,validation_data=valid_dataset,shuffle=False)
start = time.time()
model.fit(train_dataset,epochs=num_epochs,validation_data=valid_dataset,callbacks=callbacks,shuffle=False )
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/15
1078/1078 [==============================] - ETA: 0s - loss: 0.6990 - dice_coef: 0.3010 - iou: 0.1823 - accuracy: 0.0021
Epoch 1: val_loss improved from inf to 0.92591, saving model to /kaggle/working/Increased_depth_Feature1.h5
1078/1078 [==============================] - 395s 350ms/step - loss: 0.6990 - dice_coef: 0.3010 - iou: 0.1823 - accuracy: 0.0021 - val_loss: 0.9259 - val_dice_coef: 0.0739 - val_iou: 0.0395 - val_accuracy: 2.0911e-04 - lr: 0.0100
Epoch 2/15
1078/1078 [==============================] - ETA: 0s - loss: 0.5984 - dice_coef: 0.4016 - iou: 0.2577 - accuracy: 0.0031
Epoch 2: val_loss improved from 0.92591 to 0.62334, saving model to /kaggle/working/Increased_depth_Feature1.h5
1078/1078 [==============================] - 239s 221ms/step - loss: 0.5984 - dice_coef: 0.4016 - iou: 0.2577 - accuracy: 0.0031 - val_loss: 0.6233 - val_dice_coef: 0.3771 - val_iou: 0.2412 - val_accuracy: 0.0023 - lr: 0.0100
Epoch 3/15
1078/1078 [==============================] - ETA: